In [1]:
import os
import time
from __future__ import print_function
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential, load_model
from keras.callbacks import Callback, TensorBoard, ModelCheckpoint, CSVLogger
import matplotlib.pylab as plt

Using TensorFlow backend.


## TensorBoard Callback Helper Class

In [2]:
class Batched_TensorBoard(TensorBoard):

    def __init__(self, log_dir = './log_dir'):
        self.log_dir = log_dir
        self.batch_writer = tf.summary.FileWriter(self.log_dir)
        self.step = 0
        super().__init__(self.log_dir)


    def on_batch_end(self, batch, logs={}):
        """Called after every batch"""
        for name, value in logs.items():
            if name in ['acc', 'loss']:
                summary = tf.Summary()
                summary_value = summary.value.add() #Empty
                summary_value.simple_value = value.item() # 0.87 (Accuracy Value)
                summary_value.tag = name #if "acc", tag = "accuracy" for more defined tags on the tensorboard
                self.batch_writer.add_summary(summary, self.step) 
    
        self.batch_writer.flush()
        self.step += 1 # Iterated over every batch

## Initialize Globals

In [5]:
timestamp = time.strftime("%m-%d.%H:%M")
ROOT_DIR = './'
TENSORBOARD_PATH = ROOT_DIR + 'graphs/' + timestamp
CSV_DIR = ROOT_DIR + 'csv'
CSV_PATH = CSV_DIR + '/'+ timestamp + '.csv'
MODEL_DIR = ROOT_DIR + 'models' + '/' + timestamp
MODEL_PATH = MODEL_DIR + '/model.epoch.{epoch:02d}.hdf5'

In [6]:
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
if not os.path.exists(CSV_DIR):
    os.makedirs(CSV_DIR)

## Load Data

In [13]:
# laod data
img_x, img_y = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)
print('data type:', x_train.dtype)

x_train shape: (60000, 28, 28)
y_train shape: (60000,)
x_test shape: (10000, 28, 28)
y_test shape: (10000,)
data type: uint8


In [14]:
'''
# un-comment when testing changes
# only trains on 100 instances
x_train = x_train[:100]
y_train = y_train[:100]
x_test = x_test[:100]
y_test = y_test[:100]
'''

'\n# un-comment when testing changes\n# only trains on 100 instances\nx_train = x_train[:100]\ny_train = y_train[:100]\nx_test = x_test[:100]\ny_test = y_test[:100]\n'

## Data Pre Processing

In [15]:
# reshape the data into a 4D tensor - (sample_number, x_img_size, y_img_size, num_channels)
# because the MNIST is greyscale, we only have a single channel - RGB colour images would have 3
x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1)
x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 1)
input_shape = (img_x, img_y, 1)

In [16]:
# data values are in range [0, 255]
# convert to floats between [0, 1]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [17]:
# convert labels to one hot encoding
# categorical_crossentropy loss below
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Build / Compile Model

In [18]:
# resolve duplicate tensorboard computation graphs
from keras import backend as K  
K.clear_session()

In [19]:
# build CNN
model = Sequential()

# conv: 32 5x5 filters
# 28 x 28 x 1 --> 24 x 24 x 32
model.add(Conv2D(
    32,
    kernel_size=(5, 5),
    strides=(1, 1),
    activation='relu',
    input_shape=input_shape
))

# max pool:
# 24 x 24 x 32 --> 12 x 12 x 32
model.add(MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

# conv: 64 5x5 filters
# 12 x 12 x 32 --> 8 x 8 x 64
model.add(Conv2D(
    64,
    kernel_size=(5, 5),
    activation='relu'
))

# max pool:
# 8 x 8 x 64 --> 4 x 4 x 64
model.add(MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

# flatten for dense layer
model.add(Flatten())

# dense layer
model.add(Dense(1000, activation='relu'))

# softmax layer
model.add(Dense(num_classes, activation='softmax'))

# compile model
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

## Define Training Callbacks

In [22]:
# define callbacks

# helper class for tracking accuracy
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        
# AccuracyHistory callback stores per epoch accuracy on results.__dict__['history']
historyCallback = AccuracyHistory()

# TensorBoard callback writes computation graph and per batch loss
tensorboardCallback = Batched_TensorBoard(log_dir=TENSORBOARD_PATH)

# Model Checkpoint Callback saves model and weights after each epoch
modelCheckpointCallback = ModelCheckpoint(filepath=MODEL_PATH)

# CSV Checkpoint writes accuracy and loss to csv file after each epoch
csvLogCallback = CSVLogger(filename=CSV_PATH)

## Train Model

In [23]:
# define hyper params
batch_size = 128
epochs = 10

validation_data = (x_test, y_test)

results = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=validation_data,
    callbacks=[historyCallback, tensorboardCallback, modelCheckpointCallback, csvLogCallback]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 481s 8ms/step - loss: 0.1591 - acc: 0.9511 - val_loss: 0.0348 - val_acc: 0.9884
Epoch 2/10
60000/60000 [==============================] - 420s 7ms/step - loss: 0.0422 - acc: 0.9869 - val_loss: 0.0384 - val_acc: 0.9866
Epoch 3/10
60000/60000 [==============================] - 420s 7ms/step - loss: 0.0292 - acc: 0.9909 - val_loss: 0.0249 - val_acc: 0.9916
Epoch 4/10
60000/60000 [==============================] - 421s 7ms/step - loss: 0.0205 - acc: 0.9935 - val_loss: 0.0345 - val_acc: 0.9884
Epoch 5/10
60000/60000 [==============================] - 417s 7ms/step - loss: 0.0167 - acc: 0.9950 - val_loss: 0.0266 - val_acc: 0.9916
Epoch 6/10
60000/60000 [==============================] - 417s 7ms/step - loss: 0.0128 - acc: 0.9960 - val_loss: 0.0264 - val_acc: 0.9911
Epoch 7/10
60000/60000 [==============================] - 417s 7ms/step - loss: 0.0110 - acc: 0.9962 - val_loss: 0.0291 - 

## Load Saved Model

In [26]:
new_model = load_model('./models/09-27.23:19/model.epoch.10.hdf5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1025000   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10010     
Total para

In [28]:
new_model.__dict__

{'name': 'sequential_2',
 'trainable': True,
 '_is_compiled': True,
 '_expects_training_arg': False,
 '_initial_weights': None,
 'supports_masking': False,
 'optimizer': <keras.optimizers.Adam at 0xb35800240>,
 '_updates': [],
 '_losses': [],
 '_per_input_losses': {},
 '_per_input_updates': {},
 '_layers': [<keras.engine.input_layer.InputLayer at 0xb343e0550>,
 '_outbound_nodes': [],
 '_inbound_nodes': [<keras.engine.base_layer.Node at 0xb343e0f28>],
 '_is_graph_network': True,
 '_uses_inputs_arg': True,
 'outputs': [<tf.Tensor 'dense_2_1/Softmax:0' shape=(?, 10) dtype=float32>],
 'inputs': [<tf.Tensor 'conv2d_1_input_1:0' shape=(?, 28, 28, 1) dtype=float32>],
 '_built': True,
 '_compute_previous_mask': True,
 '_input_layers': [<keras.engine.input_layer.InputLayer at 0xb343e0550>],
 '_output_layers': [<keras.layers.core.Dense at 0xb355922b0>],
 '_input_coordinates': [(<keras.engine.input_layer.InputLayer at 0xb343e0550>,
   0,
   0)],
 '_output_coordinates': [(<keras.layers.core.Dense 